# Workshop: UK Continental Shelf Basemap

---
## 1. Objectives

### 1.1 Reinventing the basemap  
In this notebook we explore the process of making an interactive E&P basemap for the UK Continental Shelf.

A basemap is important for placing many types of E&P data, like well locations, seismic surveys, license blocks and field areas, in their correct spatial context. Historically, large printed E&P basemaps were a standard wall covering in many company offices.  

Reflecting the changing times and increasing digitalization, our task in this workshop is to reinvent the E&P basemap using Python code and open data sources provided by the UK Oil and Gas Authority. 

The task of creating the basemap can be posed as an exercise in **computational thinking** - the process of thinking about a problem and developing a solution in a form that can be solved by a computer. 


### 1.2 Problem statement  

Let's start by stating the problem as a set of requirements:

#### Display  
The map must display the following objects:

- **Quadrants**. Quadrants are 1x1 degree areas that subdivide the UKCS into a regular grid.  

- **Licence blocks**. Licences blocks are areas defined by polygons within which the licence holder is granted permission to explore for resources.

- **Field determinations**. Determinations are regularized boundaries that capture the maximum extent of a field and are integral to gaining consent for a field development.  

- **Field outlines**. Field outlines are irregular polygons that define the extent of a hydrocarbon accumulation. 

- **Cultural/satellite background**. Underlying the basemap there should be cultural/satellite imagery showing important geographic features.

#### Interactivity  
The map must be interactive with the following features:

- **Pan and zoom**. Users should be able to pan to navigate, and zoom to focus on details.  

- **Multi-resolution**. The map must be multi-resolution, meaning that the objects are renderered at a level of detail that depends on the amount of zoom. By zooming right in, users should be able to see the finest details in the displayed objects.  

- **Legend**. The map must have a legend that allows users to hide/show the displayed objects.  

- **Hover**. There should be a way of viewing more detail about any of the objects by simply hovering the mouse over an object.

#### Reuse, sharing, and extensibility  
The map must support reuse, sharing, and extensibility as follows:

- **Reuse**. Users should be able to reuse the basemap including all its interactive features in other notebooks without the need to run the code in this notebook again.  

- **Sharing**. Users should be able to share the basemap with colleagues by simply sharing a file.  

- **Extensibility**. Users should be able to extend the basemap by adding more objects such as the surface locations of wells, or the location of seismic survey lines. Basic knowledge of Python and the packages used in this notebook should be all that is necessary to achieve this.  

#### Data sources
The map must be built using data from the following sources:

- **GeoJSON data**. Input data for the basemap comes in the form of GeoJSON files from the Oil and Gas Authority. As you should know by now, GeoJSON is just a standard representation of geospatial data expressed in plain text JSON format. You should also recall that JSON format looks similar to the Python code that creates the objects.  

- **Tabular data**. Input data to extend the basemap with well and seismic lines come from (`.csv`) files in tabular form. As you should know by now, these are plain text files that we can read in to `pandas` data frames.  

At first sight this may seem like a very challenging set of requirements, but by applying some computational thinking we can break down the problem into manageable pieces. It also helps considerably to know that the Plotly graphics package that we're going to use provides many of the parts we need for basemap display, interactivity, and reuse.

In the following sections we start by applying computational thinking to sketch out a solution to these requirements, before writing code that implements the basemap using what we have learned in Python 101, 102.  

**Note**: Some cells in this notebook require that you complete the code in order to make progress. If you get stuck, there are solutions in:

> `workshop-ukcs-basemap-solution.ipynb`  

---
## 2. Developing a solution

### 2.1 Recall of computational thinking  

To begin let's recall the main elements of computational thinking:

- **Decomposition**. Break a complex task down into smaller, more manageable parts.

- **Patterns**. Identify and utilize reusable solutions to commonly occurring problems.

- **Representation**. Represent the problem in terms of key data objects, bypassing irrelevant details.

- **Algorithms**. Express solutions as instructions or recipes using flow charts or pseudocode.

Now lets list some initial thoughts on how to apply these ideas to the problem at hand.


### 2.2 Decomposition

We can decompose the problem into a number of smaller parts as follows:

- **Data import**. The input data comes in the form of GeoJSON (`.geojson`) and tabular (`.csv`) files - when reading these files we need to consider how best to represent the data they contain in terms of Python objects. An important factor in this decision is any constraint imposed by the display - we must be able to provide the data objects in the correct form for display. In an ideal world we want to use the data objects directly in the form they are read from file. In the basemap it turns out that most of the data **can** be used directly, but as noted in Section 2.4 it will also be necessary to modify the GeoJSON data for display.  

- **Graphics**. The basemap problem is demanding in terms of the requirements for the display and all the interactive features specified in the problem statement. We need to decide how best to make the graphics display, but luckily the Plotly package can handle these requirements. In particular we will use Plotly's Mapbox features to provide the background cultural/satellite imagery and interactivity.   

- **Sharing**. We need to provide a way to share the basemap display. One way is the export the basemap as a (`.html`) file - we can send this file to others to view in their own web browser - in contrast to a static image this preserves the interactive features.  

- **Reuse**. To reuse the basemap at a later date, we can export the basemap as a (`.json`) file containing a definition of the map. This allows us to easily recreate the basemap in a Python notebook without knowing anything about the code that was used to make the basemap in the first place.  

- **Extensibility**. The process of extending the basemap, by adding new display elements like wells and seismic survey lines, is made easy .



### 2.3 Patterns

A recurring pattern we need in our solution is:

1. Read objects from an input (`.geojson`) data file.  

2. Convert objects to a representation that can be displayed in the basemap.   

3. Add objects to the basemap.  

4. Adjust the styling (colours, opacity, lines etc) of the object display to suit our preferences.  

This is a pattern we need to repeat for each type of object - quadrants, licence blocks, and field determinations, field areas. The order in which we add objects to the basemap is important as later objects overlay (and can obscure) earlier objects. A good choice for this problem is to add objects in the following order:

1. Quadrants.  

2. Licence blocks.  

3. Field determinations.  

4. Field areas.

and when we come to extend the map later on, we can add:

5. Well surface locations.  

6. Seismic survey lines.

### 2.4 Representation of data objects

- **GeoJSON data**. The GeoJSON input data is really convenient for this project because it easiliy represents the complex data structures comprising both nested geometry and property information. One complicating factor is that we will need to change the representation of this data to create a *flattened* or tabular version of the object properties. This is a constraint imposed by the way the Plotly Mapbox display expects to receive data for a filled polygon (or choropleth) display.  

- **Flattening**. In the same folder as this notebook is a module called `flatten_geojson.py` which defines a function for performing this flattening operation. It takes an object as its argument and returns a `pandas` data frame containing the properties in a tabular form. This is simply an alternative representation of the object properties, the data values themselves remain unchanged.  

- **Tabular data**. The well data is provided in a (`.csv`) file, and after being loaded into a `pandas` data frame can be directly displayed on the basemap without any modification.  

The same remark applies to the seismic line CDP locations. This data is also provided in a (`.csv`) file and once in a `pandas` data frame can be directly displayed on the basemap.


### 2.5 Representation of basemap graphics

Another question of representation concerns how best to represent the basemap graphics in order to satisfy the interactivity, reuse, sharing, and extensibility requirements. There are several options available, as follows:

- **Pixel-based image**. The basemap graphics can be saved to a pixel-based image and stored in variety of formats like (`.png`) or (`.jpg`) using the `write_image()` function provided by Plotly. One limitation is that this representation fails on the multi-resolution display requirement as the image will appear pixelated beyond a certain zoom level. In fact, this representation fails to satisfy most of the requirements outlined in the problem statement.  

- **HTML file**. The basemap graphics can be saved to a (`.html`) file using the `write_html()` function provided by Plotly. This satisfies the multi-resolution display requirement along with the requirements for interactivity and sharing. The sharing requirement is easily satisfied by simply sharing the (`.html`) file with a colleague e.g., by email. One limitation is that it does not satisfy the extensibility requirement, as it is not suited to making further modifications in a notebook.  

- **JSON file**. The basemap graphics can be saved to a (`.json`) file using the `write_json()` function provided by Plotly. This representation can be read in a notebook to recreate the basemap and make further additions, satisfying the requirements for reuse and extensibility. The file can also be shared with colleagues so that they can also recreate and manipulate the basemap in their own notebooks.  

---
## 3. Getting  started


The following cells import the modules we are going to need, and read in an access token (a licence key) from a file so that we can use satellite imagery as a background to our basemap.

#### Cell 3.1 Import the required modules

Import the modules used in this notebook. 

In [ ]:
import os
import numpy as np
import pandas as pd
import plotly.graph_objects as go 
import plotly.io as pio

#### Cell 3.2 Read the Mapbox access token

We need an access token so that we can use Mapbox's satellite and cultural imagery in our basemap. A token is provided with these course materials in the `assets` folder.

In [ ]:
#
# read the mapbox access token to enable 'satellite' and 'satellite-streets' styles:
#
with open('../../assets/mapbox-access-token.txt') as f:
    token = f.read()

#### Cell 3.3 Set the default renderer  

The `notebook` renderer works well in this basemap exercise because the graphics output expands to fill the available width.

In [ ]:
pio.renderers.default = 'notebook'

--- 
## 4. Represent input data as Python objects 

#### Cell 4.1 Read quadrants dataset

In the same folder as this notebook, you will find the file `OGA_Quadrants_WGS84.geojson` containing the quadrant data for the UKCS. Load the objects from file and assign the result to a variable called `quads`.  

Hint: you saw how to do this in Python 102. Since this is JSON data you will need to start with an `import json` statement.

In [ ]:
# type your solution here


#### Cell 4.2 Flatten the hierarchy of objects in the quadrants dataset

The code in this cell imports the `flatten_geojson()` function and uses it to make a data frame with one row per quadrant, and one column per quadrant property. The result is called `df_quads` and we can think of it as an alternative representation of the `quads` object.

In [ ]:
from flatten_geojson import flatten_geojson

df_quads = flatten_geojson(quads)
df_quads.info()

#### Cell 4.3 What's in the quadrants dataset?

Print the first few rows of `df_quads`.

In [ ]:
df_quads.head()

#### Cell 4.4 Read licence blocks dataset

In the same folder as this notebook, you will find the file `OGA_Licences_WGS84.geojson` containing the licence block data for the UKCS. Load the objects from file and assign the result to a variable called `licences`. Use the same pattern you used for the quadrants in Cell 4.1.

In [ ]:
# type your solution here


#### Cell 4.5 Flatten the hierarchy of objects in the licence blocks dataset

Use the `flatten_geojson()` function following the same pattern as in Cell 4.2 to make a data frame with one row per license block, and one column per license block property. Call the result `df_licences`. Just like before we can think of it as an alternative representation of the `licences` object.

In [ ]:
# type your solution here


#### Cell 4.6 What's in the licence block dataset?  

Print the first few rows of `df_licences`.

In [ ]:
df_licences.head()

#### Cell 4.7 Read offshore field determinations dataset   

In the same folder as this notebook, you will find the file `OGA_Offshore_FieldDets_WGS84.geojson` containing the field determinations data for the UKCS. Load the objects from file and assign the result to a variable called `dets`. Use the same pattern you used for the quadrants and licence blocks in cells 4.1 and 4.4.

In [ ]:
# type your solution here


#### Cell 4.8 Flatten the offshore field determinations dataset

Use the `flatten_geojson()` function following the same pattern as in Cell 4.2 to make a data frame with one row per field determination, and one column per field determination property. Call the result `df_dets`. Just like before we can think of it as an alternative representation of the `dets` object.

In [ ]:
# type your solution here


#### Cell 4.9 What's in the offshore field determinations dataset  

Print the first few rows of `df_dets`.

In [ ]:
df_dets.head()

#### Cell 4.10 Read offshore fields dataset

In the same folder as this notebook, you will find the file `OGA_Offshore_Fields_WGS84.geojson` containing the offshore fields data for the UKCS. Load the objects from file and assign the result to a variable called `fields`. Use the same pattern you used for the quadrants, licence blocks, and field determinations in cells 4.1, 4.4, and 4.7.

In [ ]:
# type your solution here


#### Cell 4.11 Flatten the offshore fields dataset 

Use the `flatten_geojson()` function following the same pattern as in Cell 4.2 to make a data frame with one row per offshore field, and one column per offshore field property. Call the result `df_fields`. Just like before we can think of it as an alternative representation of the `fields` object.

In [ ]:
# type your solution here


#### Cell 4.12 What's in the offshore fields dataset?  

Print the first few rows of `df_fields`.

In [ ]:
df_fields.head()

#### Cell 4.13 What types of offshore fields are there?

The following cell uses the `unique()` function to report the values taken by the `FIELDTYPE` variable. These values represent the dominant type of hydrocarbon produced in each field and we will use this information to colour code field areas on the basemap.

In [ ]:
df_fields.FIELDTYPE.unique()

We can also drill a bit deeper and report the number of fields of each type using the `value_counts()` function as follows:

In [ ]:
df_fields.FIELDTYPE.value_counts()

---
## 5. Creating the basemap



In this section we create the basemap using the data prepared in the previous section. 

#### Cell 5.1 Create initial basemap with quadrants

In the following cell there are 6 numbered sections that:

1. Create the initial basemap figure.  

2. Add the quadrants as filled polygons.   

3. Style the map background.  

4. Style the map legend.  

5. Add an attribution statement for the data source.  

6. Render the basemap in the cell output.  

**Instructions**

In section (2) the quadrants are added to the display using the `add_choroplethmapbox()` function. A choropleth is just another name for a filled polygon. Assign the `quads` variable to the `geojson` argument, and `df_quads.OBJECTID` to the `locations` argument. This has the effect of using the geometry provided by the `quads` object, and controlling which parts of the geometry as displayed using the quadrants listed by `df_quads.OBJECTID`. 

In section (3) use the `mapbox_style` parameter to set the background cultural/satellite imagery. There are many options available, but `"satellite-streets"` is a good choice, other nice options include `"open-street-map"` or `"outdoors"`. 

In section (4), use the `legend_title_text` parameter to give the map a title and author name.

When complete you should see the initial basemap in the output cell. Try testing the interactive features such as pan, zoom, and tilt. Also use the legend to show/hide the quadrants. With the quadrants visible, you should also try hovering the mouse over a quadrant to see its number. 

In [ ]:
#
# 1. create an empty figure:
#
fig = go.Figure()

#
# 2. add quadrants as filled polygons (for which a choropleth is just a fancy name):
#
# complete your solution by setting geojson and locations arguments
fig.add_choroplethmapbox(
    geojson=# type your solution here,
    locations=# type your solution here,
    z=np.zeros_like(df_quads.QUAD_NO),
    name='Quadrants',
    colorscale=[[0, 'LightSkyBlue'], [1, 'LightSkyBlue']],
    visible=True,
    marker_opacity=0.2,
    marker_line_width=1,
    marker_line_color='White',
    showlegend=True,
    showscale=False,
    hovertext=df_quads.QUAD_NO,
    hovertemplate='Quad %{hovertext}<extra></extra>'
)

#
# 3. set background style, default zoom level, default center, and remove plot margins to make better use of the screen space:
#
# complete your solution by setting a mapbox_style
fig.update_layout(
    mapbox_style=# type your solution here,
    mapbox_accesstoken=token,
    mapbox_zoom=4,
    mapbox_center={"lat": 56.00, "lon": -2.00},
    margin={"r":0,"t":0,"l":0,"b":0}
)

#
# 4. style the legend:
#
# complete your solution by setting legend_title_text
fig.update_layout(
    showlegend=True,
    legend_title_text=# type your solution here,
    legend_title_font_color='White',
    legend_title_font_family='Helvetica',
    legend_font_color='White',
    legend_font_family='Helvetica',
    legend_x=0.005,
    legend_y=0.995,
    legend_bgcolor='Black'
)

#
# 5. add an attribution statement for the data:
#
fig.add_annotation(
    x=+0.05,
    y=+0.005,
    showarrow=False,
    text="Contains information provided by the Oil and Gas Authority.",
    font_color='White',
    font_size=10,
    font_family='Helvetica',
    xref="paper",
    yref="paper",
    xanchor='left',
    yanchor='bottom'
)

#
# 6. show basemap:
#
fig.show()

#### Cell 5.2 Add licence blocks to basemap

In the following code cell, add the licence blocks to the display using the `add_choroplethmapbox()` function. Assign the `licences` variable to the `geojson` argument, and `df_licences.OBJECTID` to the `locations` argument. This has the effect of using the geometry provided by the `licences` object, and controlling which parts of the geometry are displayed using the licences listed by `df_licences.OBJECTID`. This is same pattern used in the previous cell.

In [ ]:
#
# add licence blocks as filled polygons as a layer on top of the quadrants:
#
# complete your solution by setting geojson and locations arguments
fig.add_choroplethmapbox(
    geojson=# type your solution here,
    locations=# type your solution here,
    z=np.zeros_like(df_licences.RNDNO),
    name='Licences',
    colorscale=[[0,'LightYellow'],[1,'LightYellow']],
    visible=True,
    marker_opacity=0.7,
    marker_line_width=0.5,
    marker_line_color='White',
    showlegend=True,
    showscale=False,
    hovertext=df_licences.BLOCKREF,
    hovertemplate='Licence %{hovertext}<extra></extra>'
)

#
# show basemap:
#
fig.show()

#### Cell 5.3 Add field determinations to basemap

In the following code cell, add the field determinations to the display using the `add_choroplethmapbox()` function. Assign the `dets` variable to the `geojson` argument, and `df_dets.OBJECTID` to the `locations` argument. This has the effect of using the geometry provided by the `dets` object, and controlling which parts of the geometry are displayed using the determinations listed by `df_dets.OBJECTID`. This is same pattern used in the previous cells.

In [ ]:
#
# add field determinations as outline polygons as a layer on top of the licence blocks:
#
# complete your solution by setting geojson and locations arguments
fig.add_choroplethmapbox(
    geojson=# type your solution here,
    locations=# type your solution here,
    z=np.zeros_like(df_dets.OBJECTID),
    name='Determinations',
    colorscale=[[0,'rgba(0,0,0,0)'],[1,'rgba(0,0,0,0)']],
    visible=True,
    marker_opacity=1,
    marker_line_width=1,
    marker_line_color='Magenta',
    legendgroup='Determinations',
    showlegend=False,
    showscale=False,
    hovertext=df_dets.FIELDNAME,
    hovertemplate='Determination %{hovertext}<extra></extra>'
)

#
# add an empty line so that determinations are indicated by a line in the map legend:
#
fig.add_scattermapbox(
    lon=[np.nan],
    lat=[np.nan],
    name='Determinations',
    marker_color='Magenta',
    legendgroup='Determinations',
    mode='lines',
    showlegend=True
)
    
#
# show basemap:
#
fig.show()

#### Cell 5.4 Add field areas to basemap

In the following code cell, we're going to add the field areas to the display using the `add_choroplethmapbox()` function just like we did for quadrants, licence blocks, and determinations. This is the same pattern used in the previous cells, except that GAS, COND and OIL fields will be added in separate steps so that we can assign a different colour to each of the three fields types.

In the following code cell, we define a dictionary called `fieldtype_colour_map` which maps the different values taken by `FIELDTYPE` to a display colour.

Fields are added to the display by looping over the dictionary, calling the `query()` function to find fields of a particular type, and calling the `add_choroplethmapbox()` function to add just those fields with the specified colour. 


In [ ]:
#
# make a colour map for 'FIELDTYPE':
#
fieldtype_colour_map = {
    'GAS':  'Salmon',
    'COND': 'NavajoWhite',
    'OIL':  'YellowGreen',
}

#
# loop over each field type (GAS,COND,OIL):
#
for key,value in fieldtype_colour_map.items():
    #
    # get all the field areas of `FIELDTYPE == key`:
    #
    df_fields_type = df_fields.query('FIELDTYPE == @key')

    #
    # add fields areas of `FIELDTYPE == key` as filled polygons with the specified colour: 
    #
    fig.add_choroplethmapbox(
        geojson=fields,
        locations=df_fields_type.OBJECTID,
        z=np.zeros_like(df_fields_type.OBJECTID),
        name=key,
        colorscale=[[0,value],[1,value]],
        visible=True,
        marker_opacity=0.8,
        marker_line_width=0.5,
        marker_line_color='white',
        showlegend=True,
        legendgroup=key,
        showscale=False,
        hovertext=df_fields_type.FIELDNAME,
        hovertemplate='Field %{hovertext}<extra></extra>'
    )

#
# show basemap:
#
fig.show()

#### Cell 5.5 Export basemap

The following cell exports the basemap in the formats (`.png`, `.html`, and `.json`) discussed in Section 2.5:  


In [ ]:
#
# create a folder for the exported basemap if it doesn't already exist:
#
if not os.path.exists('./figure-exports'):
    os.mkdir('./figure-exports')

#
# export basemap as a (.png) file:
#
fig.write_image('./figure-exports/ukcs-basemap.png',scale=4)

#
# export basemap as a (.html) file:
#
fig.write_html('./figure-exports/ukcs-basemap.html')

#
# export basemap as a (.json) file with pretty indentation:
#
fig.write_json('./figure-exports/ukcs-basemap.json',pretty=True)

**Notes**: (i) Using `scale=4` in `write_image()` is optional and boosts the image resolution so that it can be magnified more before pixelation becomes apparent; this also results in proportionally larger file sizes. (ii) Using `pretty=True` in `write_json()` is optional and aimed at making the resulting (`.json`) file more readable for human users; this does not impact the ability to reuse the basemap definition at a later date.

#### Cell 5.6 Browse the exported basemap

To verify the basemap export has worked as expected go to the `./figure-exports` folder in the same location as this notebook and open previews of the exported files by clicking on the (`.png`) and (`.html`) files that were created in the previous cell. 

Note that the (`.png`) preview displays a static image that only supports pan and zoom, and eventually becomes pixelated at higher zoom levels. In contrast the (`.html`) preview provides a multi-resolution interactive basemap. 

You can also click on the (`.json`) file to confirm that it is a plain text JSON representation of the basemap.  

---
## 6. Using the basemap

A major requirement for the basemap task is the ability to recreate the basemap by loading the (`.json`) basemap definition from file and then overlay additional information. In this section we explore how to do this with examples of:

- Adding well surface locations from the Oil and Gas Authority dataset of over 12,300 wells.  

- Adding seismic line locations from a 2d survey of the East Shetland Platform, again using data provided by the Oil and Gas Authority.  

The final step will be to write an updated definition of the UKCS basemap to a (`.json`) file, including the additional well and seismic data. This updated definition can be used as the starting point for further additions in future.

#### Cell 6.1 Read UKCS basemap from a (`.json`) file

The first step is to load your UKCS basemap from the (`.json`) file you created in the previous section.

In the following cell we read the basemap definition and display the figure to confirm that it appears as expected.


In [ ]:
#
# read the ukcs basemap from existing (.json) file:
#
fig = pio.read_json('./figure-exports/ukcs-basemap-solution.json')

#
# show figure:
#
fig.show()

#### Cell 6.2 Read well surface location data

In the same folder as this notebook is a file of well header data called `OGA_Wells_WGS84.csv`. This file contains the well surface locations and other data for over 12,000 wells on the UK Continental Shelf. You have already seen this dataset in the workshop `workshop-toolkit-sanddance`.

In the following cell, read the file contents into a data frame assigned to a variable called `df_wells`.

In [ ]:
#
# read the well header data from (.csv) file into a pandas data frame:
#
# complete your solution by reading the file into a data frame
df_wells = 

#
# report info about the columns:
#
df_wells.info()

#### Cell 6.3 What's in the well headers dataset?

The following cell reports the first few rows of the dataset. The important things to note are the that the `X` and `Y` columns represent the longitude and latitude of the surface location, and the `WELLREGNO` column gives the well name. There are many other columns in this dataset which we will revisit in the workshop `workshop-ukcs-well-headers`, but for now its just `X`,`Y`, and `WELLREGNO` that are of interest for the basemap.

The `X` and `Y` coordinates are in the WGS84 coordinate reference system, so we can use these values directly for plotting on the basemap as the convention used throughout Plotly Mapbox also WGS84.

Recall that you can also view the contents of the well headers dataset using the *Variables* button on the menu bar at the top of this editor.

In [ ]:
df_wells.head()

#### Cell 6.4 Add well surface locations to basemap

The code in the following cell adds the well locations as markers overlaying the basemap.

The `add_scattermapbox()` function layers a scatter plot of well locations on top of the basemap.

The `name` parameter is a string that controls how the well location dataset is labelled in the map legend.

The `marker_color`, `marker_size`, and `marker_opacity` parameters control the styling of the well markers. You can customize the marker appearance to your own taste by modifying these parameters.

The `text`, `textposition`, `textfont_color`, and `textfont_size` parameters control the styling of the text posted at the well marker locations. You can customize the text appearance to your own taste by modifying these parameters. Note that text overplotting is avoided by an algorithm that shows/hides text postings as a function of the zoom level. 

In [ ]:
#
# overlay well surface locations:
#
fig.add_scattermapbox(
    lon=df_wells.X,
    lat=df_wells.Y,
    name='Well surface locations',
    marker_color='OrangeRed',
    marker_size=5,
    marker_opacity=1,
    text=df_wells.WELLREGNO,
    textposition='top right',
    textfont_color='white',
    textfont_size=11,
    mode='markers+text',
    hovertemplate='Well %{text}<br>(%{lon:.6f}&deg;E,%{lat:.6f}&deg;N)<extra></extra>'
)

#
# show figure:
#
fig.show()

#### Cell 6.5 Read East Shetland Platform survey line location data

In the following code, we read a (`.csv`) file containing trace location data for 2D seismic lines in the East Shetland Platform survey.

In the same folder as this notebook is a file of cdp locations for 2D seismic lines in the East Shetland Platform survey called `esp-cdp-locations-wgs84.csv`.

In the following cell, read the file contents into a data frame assigned to a variable called `df_esp`.

In [ ]:
# complete your solution by reading the file into a data frame
df_esp = 

#### Cell 6.6 What's in the line location dataset?

The following cell reports the first few rows of the dataset. The important thing to note is that the coordinates were extracted from trace headers in a set of SEG-Y files, and were originally stored in UTM Zone 30N coordinates. The coordinates have been converted to WGS84. Both sets are available:

- UTM Zone 30N (`CDP_X`,`CDP_Y`)

- WGS84 (`CDP_LON`,`CDP_LAT`)

The convention used throughout Plotly Mapbox figures is for coordinates to be provided in WGS84, so we must use the (`CDP_LON`,`CDP_LAT`) coordinates to display the CDP locations on the basemap.

Recall that you can also view the contents of the line location dataset using the *Show variables active in jupyter kernel* button on the menu bar at the top of this editor.


In [ ]:
df_esp.head()

#### Cell 6.7 Add seismic survey lines to basemap

The code in the following cell adds the seismic survey lines as markers overlaying the basemap.

The `add_scattermapbox()` function layers a scatter plot of seismic lines on top of the basemap.

The `name` parameter is a string that controls how the seismic survey lines dataset is labelled in the map legend.

The `mode` parameter controls the appearance of the seismic lines, we use the `lines+markers` option to show the CDP markers joined with a line. 

The `line_color` and `line_width` parameters control the styling of seismic lines. The `marker_symbol`, `marker_color`, and `marker_size`, parameters control the styling of the CDP markers. You can customize their appearance to your own taste by modifying these parameters.

In [ ]:
#
# add CDP locations using the same decimated dataset:
#
fig.add_scattermapbox(
    name='East Shetland Platform Survey',
    lon=df_esp.CDP_LON,
    lat=df_esp.CDP_LAT,
    customdata=df_esp,
    hovertemplate='<b>%{customdata[0]}</b><br>Line number %{customdata[1]}<br>Trace %{customdata[3]}<br>CDP %{customdata[2]}<br>(%{lon:.6f}&deg;E, %{lat:.6f}&deg;N)<extra></extra>',
    mode='lines+markers',
    line_color='chartreuse',
    line_width=1,
    marker_symbol='circle',
    marker_color='chartreuse',
    marker_size=3,
    connectgaps=False
)

#
# show figure:
#
fig.show()

#### Cell 6.8 Export updated basemap

Finally, the following cell exports the updated basemap in `.png`, `.html`, and `.json` format:  

In [ ]:
#
# create a folder for the exported basemap if it doesn't already exist:
#
if not os.path.exists('./figure-exports'):
    os.mkdir('./figure-exports')

#
# export updated basemap as a (.png) file:
#
fig.write_image('./figure-exports/ukcs-basemap-updated.png',scale=4)

#
# export updated basemap as a (.html) file:
#
fig.write_html('./figure-exports/ukcs-basemap-updated.html')

#
# export updated basemap as a (.json) file:
#
fig.write_json('./figure-exports/ukcs-basemap-updated.json',pretty=True)

**Notes**: As before, (i) using `scale=4` in `write_image()` is optional and boosts the image resolution so that it can be magnified more before pixelation becomes apparent, and (ii) using `pretty=True` in `write_json()` is optional and aimed at making the resulting (`.json`) file more readable for human users.

---
essential-data-science-2025/workshop-ukcs-basemap.ipynb  
Copyright © 2020-2025 Analytic Signal Limited, all rights reserved